# Proyecto Optimización - Mejora Proceso Tulas

---
## Maestría en Inteligencia Analítica para la Toma de Decisiones


* **Profesor**:
    * *Andrés Medaglia*
* **Grupo 8**:
    * *Angelica Ruce*
    * *David Moreno*
    * *Nicolas Cruz*
    * *Santiago Romero*

## Formulación
---

### Parámetros
>#### **Conjuntos**
>* $I$: conjunto de Oficinas de Servicio
>* $J$: conjunto de Centros Logisticos de Correspondencia - CLC
>* $K$: conjunto de Cedes Admin Regionales - CAR

>
>#### **Parámetros**
>* $a_i:$ Demandas mensuales de envio de tulas por oficina $i \in I$
>* $r_k:$ Demandas mensuales de envio de tulas por CAR $k \in K$
>* $p_k:$ Peso promedio de una tula enviada por la CAR k $k \in K$
>* $w_{kj}:$ Costo de transporte de 1 kg de CAR k a CLC j | $k \in K$, $j \in J$
>* $h_{kj}:$ Costo de transporte de 1 kg extra de CAR k a CLC j | $k \in K$, $j \in J$
>* $c_{ij}:$ Costo de transporte de una tula enviada por la Oficina i al CLC j | $i \in I$, $j \in J$


### Variables de Decisión
>* $x_{kj}:$ Cantidad de tulas a enviar del CAR k al CLC j $k\in K$, $j\in J$   
>* $m_{kj}:$ Cantidad de kg extra a enviar del CAR k al CLC j $k\in K$, $j\in J$   
>* $y_{ij}:$ Cantidad de tulas a enviar de la Oficinas i al CLC j $i\in I$, $j\in J$   

### Restricciones
> 1. Cada oficina $i\in I$ tiene una demanda de tulas a enviar que debe ser satisfecha
>> $\sum_{j\in J} y_{ij} \geq d_i; \forall i\in I$

> 2. Cada CAR $k\in K$ tiene una demanda de tulas a enviar que debe ser satisfecha
>> $\sum_{j\in J} x_{kj} \geq r_k; \forall k\in K$

> 3. El peso máximo de las tulas de enlace no debe superar los 30Kg, en caso de requerir incrementar el peso, se crea una restricción suave.
>> $\sum_{k\in K}p_k x_{kj} + m_{kj} \geq 30$

> **Naturaleza de Variables**
>>$x_{kj}\in \mathbb{Z}^{+}$

>>$y_{ij}\in \mathbb{Z}^{+}$ 

>>$m_{kj}\geq 0 , \; \forall k\in K, j \in J$

### Función Objetivo
> Maximizar el incremento en ventas dado por la inversión en canales de mercadeo.
>> $\max{ \sum_{i\in I} a_ix_i}$

## Importación de librerías
---

In [1]:
import pulp as lp
import pandas as pd

## Creación de Parámetros
---

Importación de parámetros

In [2]:
archivo = "Soporte Proyecto Optimizacion.xlsx"
demandas_oficinas = pd.read_excel(archivo, sheet_name="Demanda Oficinas")
demandas_oficinas = demandas_oficinas.set_index(demandas_oficinas.columns[0])

demandas_car = pd.read_excel(archivo, sheet_name="Demandas CAR")
demandas_car = demandas_car.set_index(demandas_car.columns[0])

peso_tula = pd.read_excel(archivo, sheet_name="Peso Tula")
peso_tula = peso_tula.set_index(peso_tula.columns[0])

costos_kg = pd.read_excel(archivo, sheet_name="CAR-CLC_W")
costos_kg = costos_kg.set_index(costos_kg.columns[0])

costos_kg_extra = pd.read_excel(archivo, sheet_name="CAR-CLC_H")
costos_kg_extra = costos_kg_extra.set_index(costos_kg_extra.columns[0])

costos_transporte = pd.read_excel(archivo, sheet_name="Ofi-CLC")
costos_transporte = costos_transporte.set_index(costos_transporte.columns[0])

### Conjuntos

In [3]:
# Conjunto Oficinas
oficinas = pd.read_excel(archivo, sheet_name="Oficinas")
lista_oficinas = oficinas['Oficinas'].tolist()
# Conjunto CLC
clc = pd.read_excel(archivo, sheet_name="CLC")
lista_clc = clc['CLC'].tolist()
# Conjunto CAR
car = pd.read_excel(archivo, sheet_name="CAR")
lista_car = car['CAR'].tolist()

In [4]:
# Parámetros indexados en (CAR, CLC)
costos_w = costos_kg.to_dict()
costos_w = {(ciudad_interior, ciudad_exterior): valor for ciudad_exterior, ciudades in costos_w.items() for ciudad_interior, valor in ciudades.items()}
costos_w_extra = costos_kg_extra.to_dict()
costos_w_extra = {(ciudad_interior, ciudad_exterior): valor for ciudad_exterior, ciudades in costos_w_extra.items() for ciudad_interior, valor in ciudades.items()}


#Parámetros indexados en (Oficina, CLC)
costo_transporte_oficinas = costos_transporte.to_dict()
costo_transporte_oficinas = {(ciudad_interior, ciudad_exterior): valor for ciudad_exterior, ciudades in costo_transporte_oficinas.items() for ciudad_interior, valor in ciudades.items()}


#Demanda Oficinas
demanda_oficina = demandas_oficinas['Demanda'].to_dict()
for clave, valor in demanda_oficina.items():
    demanda_oficina[clave] = int(valor)

#Demanda CAR
demanda_car = demandas_car['Demanda'].to_dict()
for clave, valor in demanda_car.items():
    demanda_car[clave] = int(valor)
    
#Peso tulas
peso_tula = peso_tula['Peso promedio kg'].to_dict()
for clave, valor in peso_tula.items():
    peso_tula[clave] = int(valor)
    
#Indices
CARxCLC = [(k, j) for k in lista_car for j in lista_clc]
Oficina_x_CLC = [(i, j) for i in lista_oficinas for j in lista_clc]

In [5]:
CARxCLC

[('ARAUCA', 'ARAUCA'),
 ('ARAUCA', 'BARRANQUILA'),
 ('ARAUCA', 'BOGOTA'),
 ('ARAUCA', 'BUCARAMANGA'),
 ('ARAUCA', 'CALI'),
 ('ARAUCA', 'CUCUTA'),
 ('ARAUCA', 'FLORENCIA'),
 ('ARAUCA', 'IBAGUE'),
 ('ARAUCA', 'MANIZALES'),
 ('ARAUCA', 'MEDELLIN'),
 ('ARAUCA', 'MOCOA'),
 ('ARAUCA', 'MONTERIA'),
 ('ARAUCA', 'NEIVA'),
 ('ARAUCA', 'PASTO'),
 ('ARAUCA', 'POPAYAN'),
 ('ARAUCA', 'PROVIDENCIA'),
 ('ARAUCA', 'SAN ANDRES'),
 ('ARAUCA', 'SINCELEJO'),
 ('ARAUCA', 'TUNJA'),
 ('ARAUCA', 'VALLEDUPAR'),
 ('ARAUCA', 'VILLAVICENCIO'),
 ('ARAUCA', 'YOPAL'),
 ('BARRANQUILA', 'ARAUCA'),
 ('BARRANQUILA', 'BARRANQUILA'),
 ('BARRANQUILA', 'BOGOTA'),
 ('BARRANQUILA', 'BUCARAMANGA'),
 ('BARRANQUILA', 'CALI'),
 ('BARRANQUILA', 'CUCUTA'),
 ('BARRANQUILA', 'FLORENCIA'),
 ('BARRANQUILA', 'IBAGUE'),
 ('BARRANQUILA', 'MANIZALES'),
 ('BARRANQUILA', 'MEDELLIN'),
 ('BARRANQUILA', 'MOCOA'),
 ('BARRANQUILA', 'MONTERIA'),
 ('BARRANQUILA', 'NEIVA'),
 ('BARRANQUILA', 'PASTO'),
 ('BARRANQUILA', 'POPAYAN'),
 ('BARRANQUILA', 'PRO

In [6]:
costos_w

{('ARAUCA', 'ARAUCA'): 10613,
 ('BARRANQUILA', 'ARAUCA'): 7217,
 ('BOGOTA', 'ARAUCA'): 6952,
 ('BUCARAMANGA', 'ARAUCA'): 7308,
 ('CALI', 'ARAUCA'): 6865,
 ('CUCUTA', 'ARAUCA'): 5743,
 ('FLORENCIA', 'ARAUCA'): 11200,
 ('IBAGUE', 'ARAUCA'): 9492,
 ('MANIZALES', 'ARAUCA'): 9761,
 ('MEDELLIN', 'ARAUCA'): 8991,
 ('MOCOA', 'ARAUCA'): 7454,
 ('MONTERIA', 'ARAUCA'): 7220,
 ('NEIVA', 'ARAUCA'): 9497,
 ('PASTO', 'ARAUCA'): 6168,
 ('POPAYAN', 'ARAUCA'): 6882,
 ('PROVIDENCIA', 'ARAUCA'): 9661,
 ('SAN ANDRES', 'ARAUCA'): 10838,
 ('SINCELEJO', 'ARAUCA'): 11637,
 ('TUNJA', 'ARAUCA'): 12166,
 ('VALLEDUPAR', 'ARAUCA'): 10921,
 ('VILLAVICENCIO', 'ARAUCA'): 6960,
 ('YOPAL', 'ARAUCA'): 9564,
 ('ARAUCA', 'BARRANQUILA'): 6731,
 ('BARRANQUILA', 'BARRANQUILA'): 12106,
 ('BOGOTA', 'BARRANQUILA'): 10550,
 ('BUCARAMANGA', 'BARRANQUILA'): 6730,
 ('CALI', 'BARRANQUILA'): 6866,
 ('CUCUTA', 'BARRANQUILA'): 11173,
 ('FLORENCIA', 'BARRANQUILA'): 8407,
 ('IBAGUE', 'BARRANQUILA'): 9394,
 ('MANIZALES', 'BARRANQUILA'): 64

## Modelamiento 
---

In [7]:
#Definición del problema
problema = lp.LpProblem('Minimización_Costos', lp.LpMinimize)

In [8]:
# Variable X_kj
x = lp.LpVariable.dicts(
    name = 'ruta',
    indices = CARxCLC,
    lowBound=0,
    cat=lp.LpInteger
)

In [9]:
# Variable m_kj
#m = lp.LpVariable.dicts(
 #   name = 'ruta',
  #  indices = CARxCLC,
   # lowBound=0,
    #cat=lp.LpInteger
#)

In [10]:
# Variable Y_ij
y = lp.LpVariable.dicts(
    name = 'ruta',
    indices = Oficina_x_CLC,
    lowBound=0,
    cat=lp.LpInteger
)

### Función Objetivo

In [11]:

#problema += lp.lpSum(costos_w[k,j]*x[k,j] for k in lista_car for j in lista_clc) + lp.lpSum(costos_w_extra[k,j]*m[k,j] for k in lista_car for j in lista_clc)+ lp.lpSum(costo_transporte_oficinas[i,j]*y[i,j] for i in lista_oficinas for j in lista_clc), 'Costos minimos'
problema += lp.lpSum(costos_w[k,j]*x[k,j] for k in lista_car for j in lista_clc) + lp.lpSum(costo_transporte_oficinas[i,j]*y[i,j] for i in lista_oficinas for j in lista_clc), 'Costos minimos'




### Restricciones

In [12]:
#Demanda Oficina
for i in lista_oficinas:
    problema += lp.lpSum(y[i,j] for j in lista_clc) >= demanda_oficina[i]

In [13]:
#Demanda CAR
for k in lista_car:
    problema += lp.lpSum(x[k,j] for j in lista_clc) >= demanda_car[k]

In [14]:
#No superar peso máximo ~ Restricción suave
for j in lista_clc:
    problema += lp.lpSum(peso_tula[k]*x[k,j] for k in lista_car) >= 30 

### Invocar el optimizador


In [15]:
problema.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/santiagoromero/anaconda3/envs/Maestria/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/cw/dvczbrmn4d7g1pplvzvyvc2h0000gn/T/f8a41cd753d0414f8d3f4c2f411bfa24-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/cw/dvczbrmn4d7g1pplvzvyvc2h0000gn/T/f8a41cd753d0414f8d3f4c2f411bfa24-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 839 COLUMNS
Duplicate row C0000040 at line 4455 <     X0000903  C0000040   1.000000000000e+00 >
Duplicate row C0000790 at line 4456 <     X0000903  C0000790   1.000000000000e+00 >
Duplicate row C0000812 at line 4457 <     X0000903  C0000812   5.000000000000e+00 >
Duplicate objective at line 4458 <     X0000903  OBJ        1.061300000000e+04 >
Duplicate row C0000040 at line 4467 <     X0000905  C0000040   1.000000000000e+00 >
Duplicate row C0000790 at line 4468 <     X0000905  C0000790   1.00

PulpSolverError: Pulp: Error while executing /Users/santiagoromero/anaconda3/envs/Maestria/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc